# 再生時間ごとのクリック率検定ノートブック

指定されたインプットファイル（`playback_seconds`, `imp`, `click`列を含むCSV）を読み込み、`ratio = click / imp`を計算して再生時間ごとの差が統計的に有意かをt検定で確認します。

> 仮想環境を有効化した状態で `jupyter notebook` を起動し、このノートブックを開いてください。

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats

## 1. データ読み込み
- `data_path` を入力ファイルに差し替えてください。
- `ratio = click / imp` を算出します。

In [ ]:
data_path = Path("data/sample_clicks.csv")  # ここを入力ファイルに置き換える

df = pd.read_csv(data_path)
df["ratio"] = df["click"] / df["imp"]

print(df.head())

## 2. 再生時間別の要約統計
再生時間ごとに件数・平均・標準偏差を確認します。

In [ ]:
summary = (
    df.groupby("playback_seconds")["ratio"]
    .agg(["count", "mean", "std"])
    .reset_index()
    .sort_values("playback_seconds")
)
summary

## 3. 分布の可視化
箱ひげ図で再生時間ごとのクリック率分布を可視化します。

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 4))
ax = sns.boxplot(data=df, x="playback_seconds", y="ratio")
ax.set_xlabel("再生時間（秒）")
ax.set_ylabel("クリック率 (click / imp)")
ax.set_title("再生時間別クリック率の分布")
plt.show()

## 4. Welchのt検定（対比較）

再生時間の全ての組み合わせについてWelchのt検定（等分散を仮定しない）を実施します。
多重比較の影響を抑えるため、ここではBonferroni補正後の有意水準を併記します。

In [ ]:
def welch_t_test_by_group(df: pd.DataFrame, group_col: str, value_col: str) -> pd.DataFrame:
    groups = sorted(df[group_col].unique())
    results = []
    alpha = 0.05
    n_tests = len(groups) * (len(groups) - 1) / 2
    adjusted_alpha = alpha / n_tests if n_tests else alpha

    for i, g1 in enumerate(groups):
        vals1 = df.loc[df[group_col] == g1, value_col]
        for g2 in groups[i + 1 :]:
            vals2 = df.loc[df[group_col] == g2, value_col]
            t_stat, p_value = stats.ttest_ind(vals1, vals2, equal_var=False)
            results.append({
                "group_1": g1,
                "group_2": g2,
                "t_stat": t_stat,
                "p_value": p_value,
                "significant (alpha=0.05)": p_value < alpha,
                "significant (bonferroni)": p_value < adjusted_alpha,
            })

    result_df = pd.DataFrame(results)
    result_df.attrs["adjusted_alpha"] = adjusted_alpha
    return result_df

pairwise_results = welch_t_test_by_group(df, "playback_seconds", "ratio")
print(f"Bonferroni補正後の有意水準: {pairwise_results.attrs['adjusted_alpha']:.4f}")
pairwise_results

### 判定の読み方
- `significant (alpha=0.05)`: 単純に5%水準で差があるか。
- `significant (bonferroni)`: 多重比較を考慮した有意水準で差があるか。